In [1]:
import sys
import pandas as pd
sys.path.insert(0, '..')
from featurizer import *
from helpers.utils import *

config = '../configs/config_emily.yml'

### Deduplicate

In [ ]:
make_dir('outputs/featurizer')

featurizer = Featurizer(config)

cdr = pd.read_csv('../synthetic_data/cdr.csv')
recharges = pd.read_csv('../synthetic_data/recharges.csv')

assert featurizer.cdr.count() == len(cdr)

featurizer.cdr = featurizer.cdr.union(featurizer.cdr)
featurizer.recharges = featurizer.recharges.union(featurizer.recharges)
assert featurizer.cdr.count() == 2*len(cdr)
assert featurizer.recharges.count() == 2*len(recharges)

featurizer.deduplicate()
assert featurizer.cdr.count() == len(cdr)
assert featurizer.recharges.count() == len(recharges)

### Filter Dates

In [ ]:
featurizer = Featurizer(config)

for df in [featurizer.cdr, featurizer.recharges, featurizer.mobiledata, featurizer.mobilemoney]:
    assert df.agg({'day':'min'}).collect()[0][0] == pd.to_datetime('2020-01-01')
    assert df.agg({'day':'max'}).collect()[0][0] == pd.to_datetime('2020-02-29')

featurizer.filter_dates('2020-01-05', '2020-02-01')
for df in [featurizer.cdr, featurizer.recharges, featurizer.mobiledata, featurizer.mobilemoney]:
    assert df.agg({'day':'min'}).collect()[0][0] == pd.to_datetime('2020-01-05')
    assert df.agg({'day':'max'}).collect()[0][0] == pd.to_datetime('2020-02-01')

### Remove Spammers

In [ ]:
# Test for number of spammers identified

for spammer_threshold in [0, 1.5, 2]:
    
    featurizer = Featurizer(config)

    spammers = len(featurizer.remove_spammers(spammer_threshold=spammer_threshold))
    
    cdr = pd.read_csv('../synthetic_data/cdr.csv')
    cdr['timestamp'] = pd.to_datetime(cdr['timestamp'])
    cdr['day'] = cdr['timestamp'].dt.date
    grouped = cdr.groupby(['caller_id', 'txn_type']).agg({'day':['count', 'nunique']}).reset_index()
    assert len(grouped[grouped['day']['count'] > spammer_threshold*grouped['day']['nunique']]['caller_id'].unique()) == spammers

In [ ]:
# Test for removal of spammers

spammer_threshold = 1

featurizer = Featurizer(config)

spammers = featurizer.remove_spammers(spammer_threshold=spammer_threshold)

for df in [featurizer.cdr, featurizer.recharges, featurizer.mobiledata, featurizer.mobilemoney]:
    
    caller_ids = [item[0] for item in featurizer.cdr.select('caller_id').collect()]
    assert set(spammers).intersection(set(caller_ids)) == set()
    
    if 'recipient_id' in df.columns:
        recipient_ids = [item[0] for item in featurizer.cdr.select('caller_id').collect()]
        assert set(spammers).intersection(set(recipient_ids)) == set()

### Filter Outlier Days

In [ ]:
# Test for number of days identified

for num_sds in [1, 1.5, 2]:
    featurizer = Featurizer(config)

    outliers = [pd.to_datetime(item).tz_localize(None) for item in featurizer.filter_outlier_days(num_sds)]

    cdr = pd.read_csv('../synthetic_data/cdr.csv')
    cdr['timestamp'] = pd.to_datetime(cdr['timestamp'])
    cdr['day'] = cdr['timestamp'].dt.floor('d')
    grouped = cdr.groupby('day', as_index=False).agg('count')
    u = grouped['txn_type'].mean() + num_sds*grouped['txn_type'].std()
    l = grouped['txn_type'].mean() - num_sds*grouped['txn_type'].std()
    outliers_manual = grouped[(grouped['txn_type'] < l) | (grouped['txn_type'] > u)]['day'].astype('object')
    assert list(outliers_manual.unique()) == outliers

In [ ]:
# Test for removal of days

num_sds = 2

featurizer = Featurizer(config)

original_counts = []
for df in [featurizer.cdr, featurizer.recharges, featurizer.mobiledata, featurizer.mobilemoney]:
    original_counts.append(df.select('day').distinct().count())
    
num_outliers = len(featurizer.filter_outlier_days(num_sds))

for d, df in enumerate([featurizer.cdr, featurizer.recharges, featurizer.mobiledata, featurizer.mobilemoney]):
    assert(df.select('day').distinct().count() == original_counts[d] - num_outliers)

### Diagnostic Statistics

In [ ]:
featurizer = Featurizer(config)

statistics = featurizer.diagnostic_statistics('test_output')

for fname, name in [('cdr', 'CDR')]:
    
    df = pd.read_csv('../synthetic_data/' + fname + '.csv')
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    assert((df['timestamp'].max() - df['timestamp'].min()).days + 1 == statistics[name]['Days'])
    assert(len(df) == statistics[name]['Transactions'])
    assert(len(df['caller_id'].unique()) == statistics[name]['Subscribers'])
    if 'recipient_id' in df.columns:
        assert(len(df['recipient_id'].unique()) == statistics[name]['Recipients'])

### Diagnostic Plots

In [ ]:
featurizer = Featurizer(config)

featurizer.diagnostic_plots('test_output')

### CDR Features

In [2]:
featurizer = Featurizer(config)
featurizer.location_features()

Loading CDR...
Loading antennas...
Loading recharges...
Loading mobile data...
Loading mobile money...
Calculating spatial features...


/home/em/.conda/envs/cider/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
../featurizer.py:426: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  antennas = gpd.sjoin(antennas, shapefile, op='within', how='left').drop('index_right', axis=1)
../featurizer.py:426: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: 

!!
25808


In [4]:
featurizer = Featurizer(config)
featurizer.cdr_features()
featurizer.international_features()
featurizer.location_features()
featurizer.recharges_features()
featurizer.mobiledata_features()
featurizer.mobilemoney_features()
featurizer.all_features()

Loading CDR...
Loading antennas...
Loading recharges...
Loading mobile data...
Loading mobile money...
Calculating CDR features...
Calculating international features...
Calculating spatial features...


/home/em/.conda/envs/cider/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
../featurizer.py:426: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  antennas = gpd.sjoin(antennas, shapefile, op='within', how='left').drop('index_right', axis=1)
../featurizer.py:426: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: 

Calculating recharges features...
Calculating mobile data features...
Calculating mobile money features...


In [5]:
# Total number of records
cdr = pd.read_csv('../synthetic_data/cdr.csv')
outgoing = cdr.groupby('caller_id').agg('count')\
    [['txn_type']]
incoming = cdr.groupby('recipient_id').agg('count')\
    [['txn_type']]
txns = pd.concat([outgoing, incoming])
txns['caller_id'] = txns.index
txn_count = txns.groupby('caller_id').agg('sum')
txn_count['name'] = txn_count.index
feats = featurizer.features['cdr'].toPandas()\
    [['name', 'cdr_reporting__number_of_records']]
feats['cdr_reporting__number_of_records'] = feats['cdr_reporting__number_of_records']\
    .astype('float')
merged = txn_count.merge(feats, on='name', how='outer')
assert len(merged[merged['txn_type'] != merged['cdr_reporting__number_of_records']]) == 0

In [6]:
# Mean call time
cdr = pd.read_csv('../synthetic_data/cdr.csv')
txns = pd.concat([cdr[['caller_id', 'duration']].rename({'caller_id':'name'}, axis=1), 
            cdr[['recipient_id', 'duration']].rename({'recipient_id':'name'}, axis=1)])
txns = txns.groupby('name', as_index=False).agg('mean')
feats = featurizer.features['cdr'].toPandas()
feats['cdr_call_duration__allweek__allday__call__mean'] = \
    feats['cdr_call_duration__allweek__allday__call__mean'].astype('float')
merged = feats[['name', 'cdr_call_duration__allweek__allday__call__mean']]\
    .merge(txns, on='name')
assert len(merged[merged['duration'].astype('int') != \
       merged['cdr_call_duration__allweek__allday__call__mean'].astype('int')]) == 0

In [7]:
# Number of contacts
cdr = pd.read_csv('../synthetic_data/cdr.csv')
txns = cdr[cdr['txn_type'] == 'call']
txns = pd.concat([txns[['caller_id', 'recipient_id', 'duration']]\
                      .rename({'caller_id':'name', 'recipient_id':'contact'}, axis=1), 
                  txns[['caller_id','recipient_id', 'duration']]\
                      .rename({'recipient_id':'name', 'caller_id':'contact'}, axis=1)])
contacts = pd.DataFrame(txns.groupby('name')['contact'].nunique())
feats = featurizer.features['cdr'].toPandas()\
    [['name', 'cdr_number_of_contacts__allweek__allday__call']]
feats['cdr_number_of_contacts__allweek__allday__call'] = \
    feats['cdr_number_of_contacts__allweek__allday__call'].astype('int')
merged = feats.merge(contacts, on='name', how='outer')
assert len(merged[merged['contact'] != \
                  merged['cdr_number_of_contacts__allweek__allday__call']]) == 0

### International Features

In [8]:
# Number of outgoing transactions
cdr = pd.read_csv('../synthetic_data/cdr.csv')
cdr = cdr[cdr['international'] == 'international']
feats = featurizer.features['international'].toPandas()
inter = cdr.groupby('caller_id', as_index=False).agg('count')[['caller_id', 'txn_type']]\
    .rename({'caller_id':'name'}, axis=1)
merged = feats.merge(inter, on='name')
assert list(merged['international_all__recipient_id__count'].astype('int')) == \
    list(merged['txn_type'].astype('int'))

In [9]:
# Total call duration
cdr = pd.read_csv('../synthetic_data/cdr.csv')
cdr = cdr[cdr['international'] == 'international']
feats = featurizer.features['international'].toPandas()
inter = cdr.groupby('caller_id', as_index=False).agg('sum')[['caller_id', 'duration']]\
    .rename({'caller_id':'name'}, axis=1)
merged = feats.merge(inter, on='name')
assert list(merged['international_call__duration__sum'].fillna(0).astype('float')) == \
    list(merged['duration'].astype('float'))

### Mobile Data Features

In [32]:
# Number of unique days
feats = featurizer.features['mobiledata'].toPandas()
data = pd.read_csv('../synthetic_data/mobiledata.csv')
data['day'] = data['timestamp'].apply(lambda x: x[:10])
days = pd.DataFrame(data.groupby('caller_id')['day'].nunique())
days['name'] = days.index
merged = days.merge(feats, on='name')
assert len(merged[merged['day'] != merged['mobiledata_num_days']]) == 0

In [41]:
# Max volume
feats = featurizer.features['mobiledata'].toPandas()
data = pd.read_csv('../synthetic_data/mobiledata.csv')
data = data.groupby('caller_id', as_index=False).agg('max')\
    .rename({'caller_id':'name'}, axis=1)
merged = data.merge(feats, on='name')
assert list(merged['volume'].astype('int')) == \
    list(merged['mobiledata_max_volume'].astype('int'))

### Mobile Money Features

In [80]:
# Minimum outgoing p2p volume
feats = featurizer.features['mobilemoney'].toPandas()
mm = pd.read_csv('../synthetic_data/mobilemoney.csv')
mm = mm[mm['txn_type'] == 'p2p']
mm = mm.groupby('caller_id', as_index=False).agg('min')\
    .rename({'caller_id':'name'}, axis=1)
merged = mm.merge(feats, on='name', how='outer').fillna(0)
assert (merged['mobilemoney_outgoing_p2p_amount_min'].astype('float').round(1) == \
    merged['amount'].astype('float').round(1)).all()

In [81]:
# Minimum outgoing p2p volume
feats = featurizer.features['mobilemoney'].toPandas()
mm = pd.read_csv('../synthetic_data/mobilemoney.csv')
outgoing = mm[['caller_id', 'sender_balance_before']]\
    .rename({'caller_id':'name', 'sender_balance_before':'balance'}, axis=1)
incoming = mm[['recipient_id', 'recipient_balance_before']]\
    .rename({'recipient_id':'name', 'recipient_balance_before':'balance'}, axis=1)
mm = pd.concat([outgoing, incoming])
mm = mm.groupby('name', as_index=False).agg('mean')
merged = mm.merge(feats, on='name', how='outer').fillna(0)
assert (merged['mobilemoney_all_all_balance_before_mean'].astype('float').round(1) == \
    merged['balance'].astype('float').round(1)).all()

### Recharges Features

In [93]:
# Total recharge
feats = featurizer.features['recharges'].toPandas()
recharges = pd.read_csv('../synthetic_data/recharges.csv')\
    .rename({'caller_id':'name'}, axis=1)
recharges = recharges.groupby('name', as_index=False).agg('sum')
merged = recharges.merge(feats, on='name')
assert (merged['amount'].astype('float') == merged['recharges_sum'].astype('float')).all()

### Location Features

In [3]:
feats = featurizer.features['location'].toPandas()
cdr = pd.read_csv('../synthetic_data/cdr.csv')
antennas = pd.read_csv('../synthetic_data/antennas.csv')
antennas = gpd.GeoDataFrame(antennas, geometry=gpd.points_from_xy(antennas['longitude'], antennas['latitude']))
antennas.crs = {"init":"epsg:4326"}
prefectures = gpd.read_file('../synthetic_data/prefectures.geojson')
antennas = gpd.sjoin(antennas, prefectures, op='within', how='left')[['antenna_id', 'region']]
antennas['region'] = antennas['region'].fillna('Unknown')
outgoing = cdr[['caller_id', 'caller_antenna']]\
    .rename({'caller_id':'name', 'caller_antenna':'antenna_id'}, axis=1)\
    .merge(antennas, on='antenna_id', how='left')
incoming = cdr[['recipient_id', 'recipient_antenna']]\
    .rename({'recipient_id':'name', 'recipient_antenna':'antenna_id'}, axis=1)\
    .merge(antennas, on='antenna_id', how='left')
cdr = pd.concat([outgoing, incoming])
cdr['region'] = cdr['region'].fillna('Unknown')

/home/em/.conda/envs/cider/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/em/.local/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  import sys


In [19]:
print(len(cdr[cdr['region'] == 'Unknown']))

25808


In [20]:
merged['Unknown'].sum()

6003.0

In [180]:
# Unique prefectures
unique = pd.DataFrame(cdr.groupby('name')['region'].nunique())
merged = unique.merge(feats[['name', 'location_count(prefectures)']], on='name')
assert (merged['region'].astype('int') == merged['location_count(prefectures)'].astype('int')).all()

In [8]:
# Count per prefecture
counts = cdr.groupby(['name', 'region'], as_index=False).count()\
    .rename({'antenna_id':'count'}, axis=1)\
    .pivot(index='name', columns='region', values='count').fillna(0)
merged = counts.merge(feats, on='name', how='inner')
for prefecture in counts.columns:
    assert (merged[prefecture].astype('float') == merged['location_prefectures_' + prefecture].astype('float'))\
        .all(), prefecture

AssertionError: Unknown

In [15]:
pd.read_csv('../outputs/featurizer/datasets/countbyregions.csv')\
    .pivot(index='name', columns='regions', values='count')\
    .fillna(0)['Unknown']

name
AASxnSdfla    29
AEKnAnngAv    25
AGdRPdgtbK    32
AMhiFaRiXH    25
ANZEnbkmQy    16
              ..
zeBKFbSiTi    31
zgHOvuGdmZ    23
zjKDESgBgP    26
zpTweudaaD    27
zvoiMpXZyl    25
Name: Unknown, Length: 1000, dtype: int64

In [176]:
merged[merged['Unknown'] != merged['location_prefectures_Unknown']][['Unknown', 'location_prefectures_Unknown']]

,Unknown,location_prefectures_Unknown
0,5.0,29.0
1,7.0,25.0
2,4.0,32.0
3,7.0,25.0
4,2.0,16.0
...,...,...
995,7.0,31.0
996,8.0,23.0
997,6.0,26.0
998,4.0,27.0


In [170]:
merged[merged['AGOE-NYIVE'].astype('float') != merged['location_prefectures_AGOE-NYIVE'].astype('float')]

,name,AGOE-NYIVE,AGOU,AKEBOU,AMOU,ANIE,ASSOLI,AVE,BASSAR,BINAH,...,location_prefectures_AGOE-NYIVE_percent,location_prefectures_PLAINE DU MO_percent,location_prefectures_ANIE_percent,location_prefectures_EST-MONO_percent,location_prefectures_Unknown_percent,location_prefectures_KOZAH_percent,location_prefectures_BLITTA_percent,location_count(regions),location_count(prefectures),location_count(tower_id)
